In [ ]:
#For GP Regression
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels \
    import RBF, ConstantKernel, WhiteKernel, RationalQuadratic, ExpSineSquared
from pandas.plotting import scatter_matrix
import GPy
df.head()

In [ ]:
scatter_matrix(df)
plt.show()

#AT as the only input
#X1 = scale(np.array(df.drop(['PE','V','AP','RH'],1)))

In [ ]:
#length_scale=10,length_scale_bounds=(1e-2,1e2)

#Kernel = ConstantKernel(10,(1e-2,1e2))*RBF(10,(1e-2,1e2))

#clf1 = GaussianProcessRegressor(kernel=Kernel,n_restarts_optimizer=3)
#clf1.fit(X_train,y_train)
#y_pred, sigma = clf1.predict(X_test,return_std=True)

theta = np.array([0.5,0.25,0.1]) # GP4

def kernel2(data1,data2,theta,wantderiv=True,measnoise=1.):
	# Uses exp(theta) to ensure positive hyperparams
	theta = np.squeeze(theta)
	theta = np.exp(theta)
	# Squared exponential
	if np.ndim(data1) == 1:
		d1 = np.shape(data1)[0]
		n = 1
		data1 = data1*np.ones((d1,1))
		data2 = data2*np.ones((np.shape(data2)[0],1))
	else:
		(d1,n) = np.shape(data1)

	d2 = np.shape(data2)[0]
	sumxy = np.zeros((d1,d2))
	for d in range(n):
		D1 = np.transpose([data1[:,d]]) * np.ones((d1,d2))
		D2 = [data2[:,d]] * np.ones((d1,d2))
		sumxy += (D1-D2)**2*theta[d+1]

	k = theta[0] * np.exp(-0.5*sumxy) 
	#k = theta[0]**2 * np.exp(-sumxy/(2.0*theta[1]**2)) 

	#print k
	#print measnoise*theta[2]**2*np.eye(d1,d2)
	if wantderiv:
		K = np.zeros((d1,d2,len(theta)+1))
		K[:,:,0] = k + measnoise*theta[2]*np.eye(d1,d2)
		K[:,:,1] = k 
		K[:,:,2] = -0.5*k*sumxy
		K[:,:,3] = theta[2]*np.eye(d1,d2)
		return K
	else:	
		return k + measnoise*theta[2]*np.eye(d1,d2)
    

In [ ]:
k = kernel2(X_train,X_train,theta,wantderiv=False)
kstar = kernel2(X_train,X_test,theta,wantderiv=False)
kstar = np.squeeze(kstar)
kstarstar = kernel2(X_test,X_test,theta,wantderiv=False)
kstarstar = np.squeeze(kstarstar)

In [ ]:
L = np.linalg.cholesky(k)
beta = np.linalg.solve(L.transpose(), np.linalg.solve(L,y_train))


In [ ]:
#predict
f = np.dot(kstar.transpose(), beta)
v = np.linalg.solve(L,kstar)
V = kstarstar-np.dot(v.transpose(),v) 

In [ ]:
#log posterior
logp = -1*(-0.5*np.dot(y_train.transpose(),beta) - np.sum(np.log(np.diag(L))) - np.shape(X_train)[0] /2. * np.log(2*np.pi))

In [ ]:
#y_train.reshape(7654,1)
#Xtest.shape = (len(Xtest),1)

invL = np.linalg.inv(L)
invk = np.dot(invL.T,invL)
    
#mean = np.dot(kstar,np.dot(invk,y_train))
#var = kstarstar - np.diag(np.dot(kstar,np.dot(invk,kstar.T)))
#var = np.reshape(var,(100,1))